In [1]:
import pandas as pd
import datetime
import dateparser
import numpy as np

In [3]:
data = pd.ExcelFile('original_files/Monitorizacao_COVID-19_MTSSS_9_junho_2020.xlsx')
#read multiple worksheets
all_my_sheets = [pd.read_excel(data, sheet_name) for sheet_name in data.sheet_names]

In [4]:
data.sheet_names

['Baixas por Isolamento',
 'Apoio à Familia',
 'Apoio à Familia - Lançamentos',
 'Layoff – Estimativa ',
 'Layoff – Estim. - CAE,Dim,Dist',
 'Redução de Actividade TI e MOE',
 'Despedimentos coletivos',
 'Inscrições no IEFP',
 'DES - SegSocial',
 'DES - Apoio Excepcional',
 'Prorrogação RSI']

### Nº de Baixas Por Isolamento

In [5]:
#Select the relevant information, deleting the first 8 rows
df_baixas = all_my_sheets[0].iloc[6:]

In [6]:
baixas = df_baixas[df_baixas.columns[:3]]

In [7]:
# Reset indexes
baixas.reset_index(drop=True, inplace=True)

In [8]:
# Get indexes that separate tables
b = np.concatenate(( [True], pd.isnull(baixas.iloc[:,0]), [True] )) 
indexes = np.flatnonzero(b[1:] != b[:-1])

baixas_all = baixas[indexes[0]:indexes[1]]
baixas_by_disctrict = baixas[indexes[4]:indexes[5]]

In [9]:
baixas_by_disctrict

,Nº DE BAIXAS POR ISOLAMENTO,Unnamed: 1,Unnamed: 2
106,TOTAL,NaN,122568
107,AVEIRO,NaN,66161
108,BEJA,NaN,334
109,BRAGA,NaN,7232
110,BRAGANÇA,NaN,462
111,CASTELO BRANCO,NaN,293
112,COIMBRA,NaN,1428
113,ÉVORA,NaN,240
114,FARO,NaN,1383
115,GUARDA,NaN,338


In [10]:
#dropping rows with nan values
baixas_all.dropna(inplace=True)
baixas_by_disctrict.dropna(axis=1, how='any', inplace=True)

C:\Users\Ana\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Ana\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
#renaming the columns
baixas_all.columns = ['DATA', 'POR DIA', 'ACUMULADOS']
baixas_by_disctrict.columns = ['DISTRITO', 'TOTAL']
#fomatting the date 
baixas_all['DATA'] = baixas_all['DATA'].apply(dateparser.parse)

baixas_all.to_csv(r'dataframes/baixas_all.csv', index = False)
baixas_by_disctrict.to_csv(r'dataframes/baixas_by_disctrict.csv', index = False)

C:\Users\Ana\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [127]:
baixas_all

,DATA,POR DIA,ACUMULADOS
0,2020-03-01,7,7
1,2020-03-02,80,87
2,2020-03-03,103,190
3,2020-03-04,112,302
4,2020-03-05,94,396
...,...,...,...
95,2020-06-04,415,121886
96,2020-06-05,439,122325
97,2020-06-06,7,122332
98,2020-06-07,61,122393


In [129]:
baixas_by_disctrict

,DISTRITO,TOTAL
106,TOTAL,122568
107,AVEIRO,66161
108,BEJA,334
109,BRAGA,7232
110,BRAGANÇA,462
111,CASTELO BRANCO,293
112,COIMBRA,1428
113,ÉVORA,240
114,FARO,1383
115,GUARDA,338


### Lay Off - Estimativa

In [130]:
all_my_sheets[3].dropna(axis=0, how='any', inplace=True)
all_my_sheets[3].columns = ['DATA', 'Nº NISS_EE', 'Nº TRABALHADORES', 
                            'REMUNERAÇÕES DECLARADAS']
all_my_sheets[3].head(20)

,DATA,Nº NISS_EE,Nº TRABALHADORES,REMUNERAÇÕES DECLARADAS
6,2020-03-31 00:00:00,3361,72507,69174523
7,2020-04-01 00:00:00,10322,231683,225500911
8,2020-04-02 00:00:00,17109,350028,339900463
9,2020-04-03 00:00:00,22275,425287,434790415
10,2020-04-04 00:00:00,31914,551955,570960157
11,2020-04-05 00:00:00,33633,566751,583655511
12,2020-04-06 00:00:00,34060,572727,5.89001e+08
13,2020-04-07 00:00:00,39837,641731,6.54728e+08
14,2020-04-08 00:00:00,48905,729512,7.46301e+08
15,2020-04-09 00:00:00,55639,796088,8.11078e+08


### Layoff – Estim. - CAE,Dim,Dist

In [131]:
#Select the relevant information, deleting the first 8 rows
df1=all_my_sheets[4].iloc[8:]
df1.head(10)

,EEs QUE ENTREGARAM DOCUMENTO - COVID19 - Layoff Simplificado,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,EEs QUE ENTREGARAM DOCUMENTO - COVID19 - Layoff Simplificado.1,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
8,NaN,NaN,Nº NISS_EE,Nº TRABALHADORES,Feminino,Masculino,NaN,NaN,Nº NISS_EE,NaN,NaN,NaN,Nº NISS_EE,NaN
9,TOTAL,NaN,113214,1362133,703560,658573,NaN,TOTAL,113821,1.000000,NaN,TOTAL,113821,1.000000
10,NaN,PESSOAS SINGULARES COM TRABALHADORES A CARGO,5435,10308,6871,3437,NaN,até 10 trabalhadores,92619,0.813725,NaN,AVEIRO,7424,0.065225
11,A,"Agricultura, produção animal, caça, floresta e...",864,7016,2754,4262,NaN,11 a 25 trabalhadores,13108,0.115163,NaN,BEJA,1005,0.008830
12,B,Indústrias extractivas,64,1026,158,868,NaN,26 a 49 trabalhadores,4253,0.037366,NaN,BRAGA,10884,0.095624
13,C,Indústrias transformadoras,11306,300926,140303,160623,NaN,50 a 249 trabalhadores,3300,0.028993,NaN,BRAGANÇA,1243,0.010921
14,D,"Electricidade, gás, vapor, água quente e fria ...",35,644,93,551,NaN,>= 250 trabalhadores,541,0.004753,NaN,CASTELO BRANCO,1502,0.013196
15,E,"Captação, tratamento e distribuição de água; s...",246,4016,1548,2468,NaN,NaN,NaN,NaN,NaN,COIMBRA,3874,0.034036
16,F,Construção,3921,53956,7760,46196,NaN,NaN,NaN,NaN,NaN,ÉVORA,1362,0.011966
17,G,Comércio por grosso e a retalho; reparação de ...,25223,244211,119541,124670,NaN,NaN,NaN,NaN,NaN,FARO,7136,0.062695


#### DATA RELEVANT TO WORK SECTORS

In [161]:
#Select the columns that are relevant to work sectors
df_work=df1[df1.columns[0:6]]
df_work=df_work.iloc[2:23]

#Rename columns
df_work=df_work.rename(columns={'EEs QUE ENTREGARAM DOCUMENTO - COVID19 - Layoff Simplificado':'Index','Unnamed: 1':'Setor',
                       'Unnamed: 2':'Nº NISS_EE','Unnamed: 3':'Nº TRABALHADORES','Unnamed: 4':'Feminino','Unnamed: 5':'Masculino'})
#reset index
df_work=df_work.reset_index()

#drop previous index
df_work=df_work.drop(['index','Index'],axis=1)

#Distribution by companies and gender
df_companies=df_work.sort_values(by=['Nº NISS_EE'],ascending=False)

#Show data
df_companies

#export to csv
df_companies.to_csv(r'dataframes/companies_gender_data.csv', index = False)
df_work.to_csv(r'df_work.csv', index = False)

#### DATA RELEVANT TO ORGANIZATION DIMENSION

In [156]:
#Select all the columns that are relevant to organization dimension
#Select the columns that are relevant to organization dimension
df_size=df1[df1.columns[7:10]]
df_size=df_size.iloc[2:7]

# Rename columns for easier identification
df_size=df_size.rename(columns={'EEs QUE ENTREGARAM DOCUMENTO - COVID19 - Layoff Simplificado.1':'Tamanho da empresa','Unnamed: 8':'N° Empresas',
                       'Unnamed: 9':'Percentual(%)'})
df_size['Percentual(%)']=df_size['Percentual(%)']*100

#show data
df_size

#SAVE TO CSV
df_size.to_csv(r'dataframes/organization_dimension.csv', index = False)

#### DATA RELEVANT TO REGION

In [162]:
#Select the columns that are relevant to region
df_region=df1[df1.columns[11:14]]
df_region=df_region.iloc[2:23]


# Rename columns for easier identification
df_region=df_region.rename(columns={'Unnamed: 11':'Region','Unnamed: 12':'N° Empresas',
                       'Unnamed: 13':'Percentual(%)'})
df_region['Region']=df_region['Region'].replace({'AVEIRO':'Aveiro','BEJA':'Beja','BRAGA':'Braga','BRAGANÇA':'Bragança',
                                                 'CASTELO BRANCO':'Castelo Branco','COIMBRA':'Coimbra','ÉVORA':'Évora',
                                                 'FARO':'Faro','GUARDA':'Guarda','LEIRIA':'Leiria','LISBOA':'Lisboa',
                                                 'PORTALEGRE':'Portalegre','PORTO':'Porto','SANTARÉM':'Santarém',
                                                 'SETÚBAL':'Setúbal','VIANA DO CASTELO':'Viana do Castelo',
                                                 'VILA REAL':'Vila Real','VISEU':'Viseu','R.A.AÇORES':'R.A.Açores',
                                                 'R.A.MADEIRA':'R.A.Madeira'})
df_region['Percentual(%)']=df_region['Percentual(%)']*100

#Show data
df_region

#SAVE TO CSV
df_region.to_csv(r'dataframes/organization_region.csv', index = False)

### Redução de Actividade TI e MOE

In [135]:
reducao_atividade_completa=all_my_sheets[5]
# Drop rows with no info
reducao_atividade_completa.dropna(subset=reducao_atividade_completa.columns[1:13], how='all',inplace=True)
# Reset indexes
reducao_atividade_completa.reset_index(drop=True, inplace=True)

In [136]:
# Get indexes that separate tables
m = np.concatenate(( [True], pd.isnull(reducao_atividade_completa.iloc[:,0]), [True] )) 
indexes = np.flatnonzero(m[1:] != m[:-1])

In [137]:
# Separate tables according to indexes
reducao_atividade_byday = reducao_atividade_completa[indexes[0]:indexes[1]]
reducao_atividade_bydistrict = reducao_atividade_completa[indexes[2]-2:indexes[3]] #-1 if older xls
reducao_atividade_bysex = reducao_atividade_completa[indexes[4]:indexes[5]]

# Checks if there is an empty space in the first column
# if pd.isnull(reducao_atividade_completa.iat[indexes[2],1]):
# else:
# reducao_atividade_byday=pd.concat([reducao_atividade_byday,reducao_atividade_completa[indexes[2]:indexes[3]]])
# reducao_atividade_bydistrict = reducao_atividade_completa[indexes[4]-2:indexes[5]]
# reducao_atividade_bysex = reducao_atividade_completa[indexes[6]:indexes[7]]

In [158]:
# Clean byday table 
reducao_atividade_byday.columns = ['DATA', 'TI_ParagemTotal', 'TI_Reducao','TI_Total', 'PRO_TI_ParagemTotal', 'PRO_TI_Reducao','PRO_TI_Total', 'MOE_ParagemTotal', 'MOE_Reducao','MOE_Total', 'PRO_MOE_ParagemTotal', 'PRO_MOE_Reducao','PRO_MOE_Total']

# Formatting the date - sometimes some values are already of data type
date_inds=pd.notnull(pd.to_datetime(reducao_atividade_byday['DATA'],errors='coerce'))
if date_inds.index[date_inds].size > 0:
    reducao_atividade_byday['DATA'][date_inds]=reducao_atividade_byday['DATA'][date_inds].dt.strftime('%d/%b')
reducao_atividade_byday['DATA'] = reducao_atividade_byday['DATA'].apply(dateparser.parse,date_formats=['%d/%b'])

# Remove columns that dont have datetype in DATA, for example columns with the TOTAL
reducao_atividade_byday.dropna(subset=['DATA'],inplace=True)
# Reset indexes
reducao_atividade_byday.reset_index(drop=True, inplace=True)

reducao_atividade_byday.to_csv(r'dataframes/reducao_atividade_byday.csv', index = False)

C:\Users\Ana\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Ana\anaconda3\lib\site-packages\pandas\core\generic.py:8767: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
C:\Users\Ana\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

In [16]:
reducao_atividade_byday

,DATA,TI_ParagemTotal,TI_Reducao,TI_Total,PRO_TI_ParagemTotal,PRO_TI_Reducao,PRO_TI_Total,MOE_ParagemTotal,MOE_Reducao,MOE_Total,PRO_MOE_ParagemTotal,PRO_MOE_Reducao,PRO_MOE_Total
0,2020-04-01,64088,NaN,64088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-04-02,22496,NaN,22496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-04-03,12398,NaN,12398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-04-04,3558,NaN,3558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-04-05,2459,NaN,2459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-04-06,11731,NaN,11731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2020-04-07,8516,NaN,8516,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2020-04-08,6161,NaN,6161,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2020-04-09,6018,NaN,6018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2020-04-10,1937,NaN,1937,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
# Clean bysex table 
reducao_atividade_bysex=reducao_atividade_bysex.dropna(axis=1)
reducao_atividade_bysex.columns=['TipoPedido','Total','Feminino','Masculino']
reducao_atividade_bysex.reset_index(drop=True, inplace=True)

In [140]:
reducao_atividade_bysex

,TipoPedido,Total,Feminino,Masculino
0,Redução Actividade Trabalhador Independente (M...,151221,82288,68933
1,Prorrogação - Redução Actividade Trabalhador I...,111918,61406,50512
2,Redução Actividade Trabalhador Membro Orgão Es...,10863,4177,6686


In [159]:
# Clean bydistrict table 
reducao_atividade_bydistrict.reset_index(drop=True, inplace=True)
# Find columns corresponding to the total counts
checknull=reducao_atividade_bydistrict.iloc[0].notnull() 
checknull[0]=True
checknull.array
reducao_atividade_bydistrict=reducao_atividade_bydistrict.loc[:, checknull]
# Remove rows with nans
reducao_atividade_bydistrict.dropna(inplace=True)
# Rename columns
reducao_atividade_bydistrict.columns = ['Distrito', 'TI', 'PRORROGAÇÃO_TI', 'MOE', 'PRORROGAÇÃO_MOE']
# Reset indexes
reducao_atividade_bydistrict.reset_index(drop=True, inplace=True)

reducao_atividade_bysex.to_csv(r'dataframes/reducao_atividade_bysex.csv', index = False)
reducao_atividade_bydistrict.to_csv(r'dataframes/reducao_atividade_bydistrict.csv', index = False)

### Despedimentos coletivos

In [160]:
despedimentos=all_my_sheets[6]
# Drop first column which is empty
despedimentos=despedimentos.drop(despedimentos.columns[0],axis=1)
# Drop rows with nan
despedimentos.dropna(inplace=True)
despedimentos.reset_index(drop=True, inplace=True)
# Rename columns
despedimentos.columns = ['DATA', 'COLETIVOS_TOTAL', 'COLETIVOS_MICRO','TRABALHADORES_TOTAL', 'TRABALHADORES_MICRO']

despedimentos.to_csv(r'dataframes/despedimentos.csv', index = False)

In [143]:
despedimentos

,DATA,COLETIVOS_TOTAL,COLETIVOS_MICRO,TRABALHADORES_TOTAL,TRABALHADORES_MICRO
0,2020-03-01,0,0,0,0
1,2020-03-02,0,0,0,0
2,2020-03-03,1,1,4,4
3,2020-03-04,2,1,8,4
4,2020-03-05,4,3,16,12
...,...,...,...,...,...
95,2020-06-04,13,4,302,15
96,2020-06-05,16,4,362,15
97,2020-06-06,16,4,362,15
98,2020-06-07,16,4,362,15
